<a href="https://colab.research.google.com/github/Nathan-Levy/Traffic-Flow-Prediction-in-Urban-Areas/blob/main/Heat_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow import keras

In [ ]:
controller_data_I10 = pd.read_csv('Combined flows I10.csv')
controller_data_I10.dropna(how='all', inplace=True)

controller_data_I110 = pd.read_csv('Combined flows I110.csv')
controller_data_I110.dropna(how='all', inplace=True)

controller_data_US101 = pd.read_csv('Combined flows US101.csv')
controller_data_US101.dropna(how='all', inplace=True)

In [ ]:
controller_data_I10["5 Minutes"] = pd.to_datetime(controller_data_I10["5 Minutes"])
controller_data_I10.set_index("5 Minutes", inplace=True)

# Keep only the relevant column
time_flow_I10 = controller_data_I10[['THIRD Flow (Veh/5 Minutes)', 'INDIANA Flow (Veh/5 Minutes)', 'LORENA Flow (Veh/5 Minutes)', 'MARRIETTA Flow (Veh/5 Minutes)', 'SOTO Flow (Veh/5 Minutes)', 'SANATA FE / MATEO Flow (Veh/5 Minutes)', 'CENTRAL AVE Flow (Veh/5 Minutes)', 'FM 110 NB Flow (Veh/5 Minutes)', 'HOOVER 1 Flow (Veh/5 Minutes)', 'VERMONT 2 Flow (Veh/5 Minutes)', 'VERMONT 1 Flow (Veh/5 Minutes)', 'BUDLONG Flow (Veh/5 Minutes)', 'NORMANDIE 1 Flow (Veh/5 Minutes)']]

controller_data_I110["5 Minutes"] = pd.to_datetime(controller_data_I110["5 Minutes"])
controller_data_I110.set_index("5 Minutes", inplace=True)

# Keep only the relevant column
time_flow_I110 = controller_data_I110[['Mainline VDS 760650 - AMADOR Flow (Veh/5 Minutes)', 'Mainline VDS 718045 - STADUIM Flow (Veh/5 Minutes)', 'Mainline VDS 775610 - SUNSET Flow (Veh/5 Minutes)', 'Mainline VDS 718173 - THIRD Flow (Veh/5 Minutes)', 'Mainline VDS 760643 - 8TH Flow (Veh/5 Minutes)', 'Mainline VDS 774671 - 14TH STREET Flow (Veh/5 Minutes)', 'Mainline VDS 760635 - 11TH Flow (Veh/5 Minutes)', 'Mainline VDS 718166 - WASHINGTON2 Flow (Veh/5 Minutes)', 'Mainline VDS 775636 - 21ST STREET Flow (Veh/5 Minutes)', 'Mainline VDS 764037 - ADAMS Flow (Veh/5 Minutes)', 'Mainline VDS 763447 - EXPOSITION2 Flow (Veh/5 Minutes)', 'Mainline VDS 763246 - EXPOSITION1 Flow (Veh/5 Minutes)', 'Mainline VDS 718041 - KING BLVD Flow (Veh/5 Minutes)']]

controller_data_US101["5 Minutes"] = pd.to_datetime(controller_data_US101["5 Minutes"])
controller_data_US101.set_index("5 Minutes", inplace=True)

# Keep only the relevant column
time_flow = controller_data_US101[['RAMPART Flow (Veh/5 Minutes)', 'ALVARADO Flow (Veh/5 Minutes)', 'GLENDALE Flow (Veh/5 Minutes)', 'EDGEWARE Flow (Veh/5 Minutes)', 'N OF 110 Flow (Veh/5 Minutes)', 'BROADWAY Flow (Veh/5 Minutes)', 'FIRST STREET Flow (Veh/5 Minutes)', 'LOS ANGELES ST Flow (Veh/5 Minutes)', 'GAREY STREET Flow (Veh/5 Minutes)', 'CONCORD Flow (Veh/5 Minutes)', 'CALZONA Flow (Veh/5 Minutes)', 'DITMAN Flow (Veh/5 Minutes)']]


In [ ]:
from folium.plugins import HeatMapWithTime
import folium
import pandas as pd
import numpy as np
import branca.colormap as cm

# Sample time_flow DataFrame
# Ensure that your actual DataFrame has a timestamp index and valid flow data
time_flow = pd.DataFrame({
    "THIRD_I10": time_flow_I10['THIRD Flow (Veh/5 Minutes)'].tolist(),
    "INDIANA": time_flow_I10['INDIANA Flow (Veh/5 Minutes)'].tolist(),
    "LORENA": time_flow_I10['LORENA Flow (Veh/5 Minutes)'].tolist(),
    "MARRIETTA": time_flow_I10['MARRIETTA Flow (Veh/5 Minutes)'].tolist(),
    "SOTO": time_flow_I10['SOTO Flow (Veh/5 Minutes)'].tolist(),
    "SANTA_FE": time_flow_I10['SANATA FE / MATEO Flow (Veh/5 Minutes)'].tolist(),
    "CENTRAL_AVE": time_flow_I10['CENTRAL AVE Flow (Veh/5 Minutes)'].tolist(),
    "FM_110_NB": time_flow_I10['FM 110 NB Flow (Veh/5 Minutes)'].tolist(),
    "HOOVER_1": time_flow_I10['HOOVER 1 Flow (Veh/5 Minutes)'].tolist(),
    "VERMONT_2": time_flow_I10['VERMONT 2 Flow (Veh/5 Minutes)'].tolist(),
    "VERMONT_1": time_flow_I10['VERMONT 1 Flow (Veh/5 Minutes)'].tolist(),
    "BUDLONG": time_flow_I10['BUDLONG Flow (Veh/5 Minutes)'].tolist(),
    "NORMANDIE_1": time_flow_I10['NORMANDIE 1 Flow (Veh/5 Minutes)'].tolist(),
    "AMADOR": time_flow_I110['Mainline VDS 760650 - AMADOR Flow (Veh/5 Minutes)'].tolist(),
    "STADIUM": time_flow_I110['Mainline VDS 718045 - STADUIM Flow (Veh/5 Minutes)'].tolist(),
    "SUNSET": time_flow_I110['Mainline VDS 775610 - SUNSET Flow (Veh/5 Minutes)'].tolist(),
    "THIRD_I110": time_flow_I110['Mainline VDS 718173 - THIRD Flow (Veh/5 Minutes)'].tolist(),
    "8TH": time_flow_I110['Mainline VDS 760643 - 8TH Flow (Veh/5 Minutes)'].tolist(),
    "14TH": time_flow_I110['Mainline VDS 774671 - 14TH STREET Flow (Veh/5 Minutes)'].tolist(),
    "11TH": time_flow_I110['Mainline VDS 760635 - 11TH Flow (Veh/5 Minutes)'].tolist(),
    "WASHINGTON2": time_flow_I110['Mainline VDS 718166 - WASHINGTON2 Flow (Veh/5 Minutes)'].tolist(),
    "21ST_STREET": time_flow_I110['Mainline VDS 775636 - 21ST STREET Flow (Veh/5 Minutes)'].tolist(),
    "ADAMS": time_flow_I110['Mainline VDS 764037 - ADAMS Flow (Veh/5 Minutes)'].tolist(),
    "EXPOSITION2": time_flow_I110['Mainline VDS 763447 - EXPOSITION2 Flow (Veh/5 Minutes)'].tolist(),
    "EXPOSITION1": time_flow_I110['Mainline VDS 763246 - EXPOSITION1 Flow (Veh/5 Minutes)'].tolist(),
    "KING_BLVD": time_flow_I110['Mainline VDS 718041 - KING BLVD Flow (Veh/5 Minutes)'].tolist(),
    "RAMPART": time_flow['RAMPART Flow (Veh/5 Minutes)'].tolist(),
    "ALVARADO": time_flow['ALVARADO Flow (Veh/5 Minutes)'].tolist(),
    "GLENDALE": time_flow['GLENDALE Flow (Veh/5 Minutes)'].tolist(),
    "EDGEWARE": time_flow['EDGEWARE Flow (Veh/5 Minutes)'].tolist(),
    "N_OF_110": time_flow['N OF 110 Flow (Veh/5 Minutes)'].tolist(),
    "BROADWAY": time_flow['BROADWAY Flow (Veh/5 Minutes)'].tolist(),
    "FIRST_STREET": time_flow['FIRST STREET Flow (Veh/5 Minutes)'].tolist(),
    "LOS_ANGELES_ST": time_flow['LOS ANGELES ST Flow (Veh/5 Minutes)'].tolist(),
    "GAREY_STREET": time_flow['GAREY STREET Flow (Veh/5 Minutes)'].tolist(),
    "CONCORD": time_flow['CONCORD Flow (Veh/5 Minutes)'].tolist(),
    "CALZONA": time_flow['CALZONA Flow (Veh/5 Minutes)'].tolist(),
    "DITMAN": time_flow['DITMAN Flow (Veh/5 Minutes)'].tolist()
}, index=pd.date_range("2024-04-01", periods=2016, freq="5T"))

detector_locations = {
    "THIRD_I10": (34.032661, -118.182471),
    "INDIANA": (34.031713, -118.190515),
    "LORENA": (34.029648, -118.199219),
    "MARRIETTA": (34.029065, -118.211497),
    "SOTO": (34.029636, -118.217534),
    "SANATA_FE": (34.027652, -118.232238),
    "CENTRAL_AVE": (34.028251, -118.253349),
    "FM_100_NB": (34.038055, -118.279858),
    "HOOVER_1": (34.037067, -118.286318),
    "VERMONT_2": (34.03706, -118.289279),
    "VERMONT_1": (34.037096, -118.294389),
    "BUDLONG": (34.037129, -118.296691),
    "NORMANDIE_1": (34.037222, -118.302221),
    "AMADOR": (34.074472, -118.233103),
    "STADUIM": (34.068002, -118.238015),
    "SUNSET": (34.063457, -118.247963),
    "THIRD_I110": (34.055433, -118.256754),
    "8TH": (34.04828, -118.266145),
    "14TH": (34.042012, -118.273319),
    "11TH": (34.040612, -118.27373),
    "WASHINGTON2": (34.03711, -118.274257),
    "21ST_STREET": (34.031839, -118.274646),
    "ADAMS": (34.026164, -118.275402),
    "EXPOSITION2": (34.018314, -118.280127),
    "EXPOSITION1": (34.015503, -118.281106),
    "KING_BLVD": (34.009215, -118.281288),
    "RAMPART": (34.074797, -118.273463),
    "ALVARADO": (34.071462, -118.265891),
    "GLENDALE": (34.070002, -118.262279),
    "EDGEWARE": (34.067067, -118.254688),
    "N_OF_110": (34.064756, -118.251325),
    "BROADWAY": (34.056861, -118.242427),
    "FIRST_STREET": (34.056156, -118.241309),
    "LOS_ANGELES_ST": (34.053705, -118.235846),
    "GAREY_STREET": (34.053512, -118.232514),
    "CONCORD": (34.025109, -118.20517),
    "CALZONA": (34.020961, -118.195456),
    "DITMAN": (34.020088, -118.189029)
}

# Normalize flow values
max_flow = time_flow.max().max()
time_flow = time_flow / max_flow  # Scale between 0-1

# Prepare Heatmap Data
heatmap_data = []
for timestamp, row in time_flow.iterrows():
    time_slot = []
    for column, flow_value in row.items():
        detector_id = column.split()[0]
        if detector_id in detector_locations:
            lat, lon = detector_locations[detector_id]
            time_slot.append([lat, lon, flow_value])
    heatmap_data.append(time_slot)

# Flatten the data
flattened_data = []
for timestamp, time_slot in zip(time_flow.index, heatmap_data):
    for lat, lon, weight in time_slot:
        flattened_data.append([timestamp, lat, lon, weight])

# Create DataFrame
df_heatmap = pd.DataFrame(flattened_data, columns=["Timestamp", "Latitude", "Longitude", "Weight"])

# Convert Timestamp to datetime and sort
df_heatmap['Timestamp'] = pd.to_datetime(df_heatmap['Timestamp'])
df_heatmap = df_heatmap.sort_values(by="Timestamp")

# Prepare data for HeatMapWithTime
time_index = list(df_heatmap['Timestamp'].astype(str).unique())

data = []
for _, d in df_heatmap.groupby('Timestamp'):
    data.append([[row['Latitude'], row['Longitude'], row['Weight']] for _, row in d.iterrows()])

hmt = folium.Map(location=[34.028251, -118.253349],
                tiles="cartodbpositron",
                zoom_start=13,
                control_scale=True,
                max_zoom=12,
                min_zoom=13
                 )

HeatMapWithTime(data,
                index=time_index,
                auto_play=True,
                max_opacity=0.6,
                radius=15,
                use_local_extrema=False
               ).add_to(hmt)

hmt


<ipython-input-8-28ab014a6e34>:48: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  }, index=pd.date_range("2024-04-01", periods=2016, freq="5T"))
